# Learning about Jupyter in Jupyter Notebooks 🎉

Howdy! This notebook is a sample made for a VS Code Livestream (video link soon) on Sept 15th, 2022.
Some of the code here is intended to be a draft as we will use some of the cool Jupyter extension features to fix it up on stream 👍

## Let's get some data

As always, good to start with some imports.
For this demo, we will be using `ghapi`, `pandas`, `numpy`, and `matplotlib` (oh my!) 

In [ ]:
from ghapi.all import GhApi, paged
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

First, we need to create an instance of the API class.

In [ ]:
api = GhApi()


Let's call the GitHub API and store the results as a list.
The API results are paged, as some of the data you might ask for could be very long, so we just nest a list comprehension to "flip" through the pages.

In [ ]:
api_data = paged(api.repos.list_for_org, org="jupyter")
jupyter_repos = [r for page in api_data for r in page]

In [ ]:
jupyter_repos

## Process and clean the data with `pandas`

Lists of dictionaries are ok, but let's make that a `pd.DataFrame` so we can more easily work with the data.
Here is also where we can reduce the size of our data by picking out the interesting bits.

In [ ]:
interesting_info = {
    "name": "str",
    "html_url": "str",
    "description": "str",
    "homepage": "str",

"size": "int64",
    "stargazers_count": "int64",
    "watchers_count": "int64",
    "language": "str",
    "forks_count": "int64",
    "open_issues_count": "int64",
    "license": "object",
    "topics": "object",
    "default_branch": "str",
}

jupyter_df = pd.DataFrame(jupyter_repos, columns=list(interesting_info.keys()))

Now that the DataFrame is loaded, we can use the variable inspector in VS Code to look at the values, even better than printing out the value here.

Let's also use some other tools to explore the data a bit.

In [ ]:
# Show stats about numeric columns in the dataframe
jupyter_df.describe()

In [ ]:
jupyter_df.info()

In [ ]:
# The license column is an object, but how can we extract the license info...

The license column has dictionary values that would be nice to split out, and get rid of unnecessary info. We can use the autodocstring extension to help us document the function, and inlay hints can help show inferred missing type hints.

In [ ]:
def format_dict_column(data : pd.DataFrame, column : str, mapping : dict):
    split_df = pd.json_normalize(data.loc[:,column]).rename(columns = mapping)
    return data.drop(column, axis=1).join(split_df.loc[:,list(mapping.values())])

jupyter_df_clean = format_dict_column(df, "license", {"key" : "license_key", "name" : "license_name", "url" : "license_url"})

In [ ]:
import pandas_profiling
# If you only want to look at numeric columns, you can use the following to filter the dataframe
# df_numeric = jupyter_df_clean.select_dtypes(include=np.int64)
pandas_profiling.ProfileReport(jupyter_df_clean)# can use minimal=True to reduce output


## Visualizations

In [ ]:
language_totals = jupyter_df_clean.groupby("language").sum().sort_values("stargazers_count", ascending=False)

In [ ]:
language_totals.plot.barh(figsize=(20, 10), fontsize=20)

---
## Utility

Check you GitHub API limits:

In [ ]:
api.rate_limit.get()

If the API limit is reached, load from included csv:

In [ ]:
# If you need to export the dataframe to a csv file, you can use the following
# df.to_csv("jupyter_repos.csv")
pd.read_csv("jupyter_repos.csv")